# Neural network for h->aa->bbtautau signal/background separation

# For shape uncertainties: compute and save nominal/shifted NN outputs

## Packages

In [ ]:
import ROOT
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Activation, Dense, Dropout

import joblib

from root_numpy import array2root

## Retrieve saved models

In [ ]:
savedscaler_em1b = joblib.load('trained_models_rerunBtag/em1b_scaler.gz')
savedmodel_em1b = keras.models.load_model('trained_models_rerunBtag/em1b_model')
savedmodel_em1b.summary()

In [ ]:
savedscaler_em2b = joblib.load('trained_models_rerunBtag/em2b_scaler.gz')
savedmodel_em2b = keras.models.load_model('trained_models_rerunBtag/em2b_model')
savedmodel_em2b.summary()

## Define file paths

In [ ]:
############use different outfolders for nominal and shape
#2018
infolder18 = 'root_raw/em18_raw/'
outfolder18 = 'root_outputs/em18_outputs/'

inpaths18 = ['data_obs.root', 'embedded.root', 'DY.root', 'DY1.root', 'DY2.root', 'DY3.root', 'DY4.root'\
         , 'GGHTT.root', 'GGHWW.root', 'GGZHLLTT.root', 'GGZHNNTT.root'\
         , 'GGZHQQTT.root', 'GGZHWW.root', 'ST_tW_antitop.root'\
         , 'ST_tW_top.root', 'ST_t_antitop.root', 'ST_t_top.root'\
         , 'TTTo2L2Nu.root', 'TTToHadronic.root', 'TTToSemiLeptonic.root', 'VBFHTT.root', 'VBFHWW.root', 'VV2L2Nu.root'\
         , 'W.root', 'W1.root', 'W2.root', 'W3.root', 'W4.root', 'WZ2L2Q.root', 'WZ3L1Nu.root', 'WminusHTT.root', 'WminusHWW.root'\
         , 'WplusHTT.root', 'WplusHWW.root', 'ZHTT.root', 'ZHWW.root'\
         , 'ZZ2L2Q.root', 'ZZ4L.root'\
         , 'ggH_bbtt12.root', 'ggH_bbtt15.root', 'ggH_bbtt20.root', 'ggH_bbtt25.root', 'ggH_bbtt30.root', 'ggH_bbtt35.root', 'ggH_bbtt40.root'\
         , 'ggH_bbtt45.root', 'ggH_bbtt50.root', 'ggH_bbtt55.root', 'ggH_bbtt60.root', 'ttHnonbb.root', 'ttHbb.root'\
         , 'vbf_bbtt12.root', 'vbf_bbtt15.root', 'vbf_bbtt20.root', 'vbf_bbtt25.root', 'vbf_bbtt30.root', 'vbf_bbtt35.root', 'vbf_bbtt40.root'\
         , 'vbf_bbtt45.root', 'vbf_bbtt50.root', 'vbf_bbtt55.root', 'vbf_bbtt60.root']

outpaths18 = ['data_obs.root', 'embedded.root', 'DY.root', 'DY1.root', 'DY2.root', 'DY3.root', 'DY4.root'\
         , 'GGHTT.root', 'GGHWW.root', 'GGZHLLTT.root', 'GGZHNNTT.root'\
         , 'GGZHQQTT.root', 'GGZHWW.root', 'ST_tW_antitop.root'\
         , 'ST_tW_top.root', 'ST_t_antitop.root', 'ST_t_top.root'\
         , 'TTTo2L2Nu.root', 'TTToHadronic.root', 'TTToSemiLeptonic.root', 'VBFHTT.root', 'VBFHWW.root', 'VV2L2Nu.root'\
         , 'W.root', 'W1.root', 'W2.root', 'W3.root', 'W4.root', 'WZ2L2Q.root', 'WZ3L1Nu.root', 'WminusHTT.root', 'WminusHWW.root'\
         , 'WplusHTT.root', 'WplusHWW.root', 'ZHTT.root', 'ZHWW.root'\
         , 'ZZ2L2Q.root', 'ZZ4L.root'\
         , 'gghbbtt12.root', 'gghbbtt15.root', 'gghbbtt20.root', 'gghbbtt25.root', 'gghbbtt30.root', 'gghbbtt35.root', 'gghbbtt40.root'\
         , 'gghbbtt45.root', 'gghbbtt50.root', 'gghbbtt55.root', 'gghbbtt60.root', 'ttHnonbb.root', 'ttHbb.root'\
         , 'vbfbbtt12.root', 'vbfbbtt15.root', 'vbfbbtt20.root', 'vbfbbtt25.root', 'vbfbbtt30.root', 'vbfbbtt35.root', 'vbfbbtt40.root'\
         , 'vbfbbtt45.root', 'vbfbbtt50.root', 'vbfbbtt55.root', 'vbfbbtt60.root']

for i in range(len(inpaths18)):
    inpaths18[i] = infolder18 + inpaths18[i]
    outpaths18[i] = outfolder18 + outpaths18[i]
    
allfiles18 = []
for i in range(len(inpaths18)):
    allfiles18.append(ROOT.RDataFrame('emu_tree', inpaths18[i]))

#2017
infolder17 = 'root_raw/em17_raw/'
outfolder17 = 'root_outputs/em17_outputs/'

inpaths17 = ['data_obs.root', 'embedded.root', 'DY.root', 'DY1.root', 'DY2.root', 'DY3.root', 'DY4.root'\
         , 'GGHTT.root', 'GGHWW.root', 'GGZHLLTT.root', 'GGZHNNTT.root'\
         , 'GGZHQQTT.root', 'GGZHWW.root', 'ST_tW_antitop.root'\
         , 'ST_tW_top.root', 'ST_t_antitop.root', 'ST_t_top.root'\
         , 'TTTo2L2Nu.root', 'TTToHadronic.root', 'TTToSemiLeptonic.root', 'VBFHTT.root', 'VBFHWW.root', 'VV2L2Nu.root'\
         , 'W.root', 'W1.root', 'W2.root', 'W3.root', 'W4.root', 'WZ2L2Q.root', 'WZ3L1Nu.root', 'WminusHTT.root', 'WminusHWW.root'\
         , 'WplusHTT.root', 'WplusHWW.root', 'ZHTT.root', 'ZHWW.root'\
         , 'ZZ2L2Q.root', 'ZZ4L.root'\
         , 'ggH_bbtt12.root', 'ggH_bbtt15.root', 'ggH_bbtt20.root', 'ggH_bbtt25.root', 'ggH_bbtt30.root', 'ggH_bbtt35.root', 'ggH_bbtt40.root'\
         , 'ggH_bbtt45.root', 'ggH_bbtt50.root', 'ggH_bbtt55.root', 'ggH_bbtt60.root', 'ttHnonbb.root', 'ttHbb.root'\
         , 'vbf_bbtt12.root', 'vbf_bbtt15.root', 'vbf_bbtt20.root', 'vbf_bbtt25.root', 'vbf_bbtt30.root', 'vbf_bbtt35.root', 'vbf_bbtt40.root'\
         , 'vbf_bbtt45.root', 'vbf_bbtt50.root', 'vbf_bbtt55.root', 'vbf_bbtt60.root']

outpaths17 = ['data_obs.root', 'embedded.root', 'DY.root', 'DY1.root', 'DY2.root', 'DY3.root', 'DY4.root'\
         , 'GGHTT.root', 'GGHWW.root', 'GGZHLLTT.root', 'GGZHNNTT.root'\
         , 'GGZHQQTT.root', 'GGZHWW.root', 'ST_tW_antitop.root'\
         , 'ST_tW_top.root', 'ST_t_antitop.root', 'ST_t_top.root'\
         , 'TTTo2L2Nu.root', 'TTToHadronic.root', 'TTToSemiLeptonic.root', 'VBFHTT.root', 'VBFHWW.root', 'VV2L2Nu.root'\
         , 'W.root', 'W1.root', 'W2.root', 'W3.root', 'W4.root', 'WZ2L2Q.root', 'WZ3L1Nu.root', 'WminusHTT.root', 'WminusHWW.root'\
         , 'WplusHTT.root', 'WplusHWW.root', 'ZHTT.root', 'ZHWW.root'\
         , 'ZZ2L2Q.root', 'ZZ4L.root'\
         , 'gghbbtt12.root', 'gghbbtt15.root', 'gghbbtt20.root', 'gghbbtt25.root', 'gghbbtt30.root', 'gghbbtt35.root', 'gghbbtt40.root'\
         , 'gghbbtt45.root', 'gghbbtt50.root', 'gghbbtt55.root', 'gghbbtt60.root', 'ttHnonbb.root', 'ttHbb.root'\
         , 'vbfbbtt12.root', 'vbfbbtt15.root', 'vbfbbtt20.root', 'vbfbbtt25.root', 'vbfbbtt30.root', 'vbfbbtt35.root', 'vbfbbtt40.root'\
         , 'vbfbbtt45.root', 'vbfbbtt50.root', 'vbfbbtt55.root', 'vbfbbtt60.root']

for i in range(len(inpaths17)):
    inpaths17[i] = infolder17 + inpaths17[i]
    outpaths17[i] = outfolder17 + outpaths17[i]

allfiles17 = []
for i in range(len(inpaths17)):
    allfiles17.append(ROOT.RDataFrame('emu_tree', inpaths17[i]))
    
#2016
infolder16 = 'root_raw/em16_raw/'
outfolder16 = 'root_outputs/em16_outputs/'

inpaths16 = ['data_obs.root', 'embedded.root', 'DY.root', 'DY1.root', 'DY2.root', 'DY3.root', 'DY4.root'\
         , 'GGHTT.root', 'GGHWW.root', 'GGZHLLTT.root', 'GGZHNNTT.root'\
         , 'GGZHQQTT.root', 'GGZHWW.root', 'ST_tW_antitop.root'\
         , 'ST_tW_top.root', 'ST_t_antitop.root', 'ST_t_top.root'\
         , 'TTTo2L2Nu.root', 'TTToHadronic.root', 'TTToSemiLeptonic.root', 'VBFHTT.root', 'VBFHWW.root', 'VV2L2Nu.root'\
         , 'W.root', 'W1.root', 'W2.root', 'W3.root', 'W4.root', 'WZ2L2Q.root', 'WZ3L1Nu.root', 'WminusHTT.root', 'WminusHWW.root'\
         , 'WplusHTT.root', 'WplusHWW.root', 'ZHTT.root', 'ZHWW.root'\
         , 'ZZ2L2Q.root', 'ZZ4L.root'\
         , 'ggH_bbtt12.root', 'ggH_bbtt15.root', 'ggH_bbtt20.root', 'ggH_bbtt25.root', 'ggH_bbtt30.root', 'ggH_bbtt35.root', 'ggH_bbtt40.root'\
         , 'ggH_bbtt45.root', 'ggH_bbtt50.root', 'ggH_bbtt55.root', 'ggH_bbtt60.root', 'ttHnonbb.root', 'ttHbb.root'\
         , 'vbf_bbtt12.root', 'vbf_bbtt15.root', 'vbf_bbtt20.root', 'vbf_bbtt25.root', 'vbf_bbtt30.root', 'vbf_bbtt35.root', 'vbf_bbtt40.root'\
         , 'vbf_bbtt45.root', 'vbf_bbtt50.root', 'vbf_bbtt55.root', 'vbf_bbtt60.root']

outpaths16 = ['data_obs.root', 'embedded.root', 'DY.root', 'DY1.root', 'DY2.root', 'DY3.root', 'DY4.root'\
         , 'GGHTT.root', 'GGHWW.root', 'GGZHLLTT.root', 'GGZHNNTT.root'\
         , 'GGZHQQTT.root', 'GGZHWW.root', 'ST_tW_antitop.root'\
         , 'ST_tW_top.root', 'ST_t_antitop.root', 'ST_t_top.root'\
         , 'TTTo2L2Nu.root', 'TTToHadronic.root', 'TTToSemiLeptonic.root', 'VBFHTT.root', 'VBFHWW.root', 'VV2L2Nu.root'\
         , 'W.root', 'W1.root', 'W2.root', 'W3.root', 'W4.root', 'WZ2L2Q.root', 'WZ3L1Nu.root', 'WminusHTT.root', 'WminusHWW.root'\
         , 'WplusHTT.root', 'WplusHWW.root', 'ZHTT.root', 'ZHWW.root'\
         , 'ZZ2L2Q.root', 'ZZ4L.root'\
         , 'gghbbtt12.root', 'gghbbtt15.root', 'gghbbtt20.root', 'gghbbtt25.root', 'gghbbtt30.root', 'gghbbtt35.root', 'gghbbtt40.root'\
         , 'gghbbtt45.root', 'gghbbtt50.root', 'gghbbtt55.root', 'gghbbtt60.root', 'ttHnonbb.root', 'ttHbb.root'\
         , 'vbfbbtt12.root', 'vbfbbtt15.root', 'vbfbbtt20.root', 'vbfbbtt25.root', 'vbfbbtt30.root', 'vbfbbtt35.root', 'vbfbbtt40.root'\
         , 'vbfbbtt45.root', 'vbfbbtt50.root', 'vbfbbtt55.root', 'vbfbbtt60.root']

for i in range(len(inpaths16)):
    inpaths16[i] = infolder16 + inpaths16[i]
    outpaths16[i] = outfolder16 + outpaths16[i]

allfiles16 = []
for i in range(len(inpaths16)):
    allfiles16.append(ROOT.RDataFrame('emu_tree', inpaths16[i]))

## Define nominal variables for 1 bjet

In [ ]:
myele = 'ROOT::Math::PtEtaPhiMVector(pt_1,eta_1,phi_1,m_1)'
mymu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
mymet = 'ROOT::Math::PtEtaPhiMVector(met,0,metphi,0)'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mymu+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_1,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
m_btt = '(mytt+myb1).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
tt_pt = 'mytt.Pt()'
tt_eta = 'mytt.Eta()'

mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(len(files18)):
    files18[i] = files18[i].Define('myele', myele)\
    .Define('mymu', mymu)\
    .Define('mymet', mymet)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('m_btt', m_btt)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('tt_pt', tt_pt)\
    .Define('tt_eta', tt_eta)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['pt_1', 'pt_2', 'm_btt', 'm_b1ele', 'm_b1mu',\
                'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b1tt',\
                'tt_eta', 'mT_ele', 'mT_mu', 'mT_b1', 'Dzeta',\
                'bpt_deepflavour_1']

## Define nominal variables for 2 bjets

In [ ]:
myele = 'ROOT::Math::PtEtaPhiMVector(pt_1,eta_1,phi_1,m_1)'
mymu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
mymet = 'ROOT::Math::PtEtaPhiMVector(met,0,metphi,0)'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mymu+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_1,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
myb2 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_2,beta_deepflavour_2,bphi_deepflavour_2,bm_deepflavour_2)'
m_b1tt = '(mytt+myb1).M()'
m_b2tt = '(mytt+myb2).M()'
m_bbtt = '(mytt+myb1+myb2).M()'
m_bb = '(myb1+myb2).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
m_b2ele = '(myele+myb2).M()'
m_b2mu = '(mymu+myb2).M()'
m_bbele = '(myb1+myb2+myele).M()'
m_bbmu = '(myb1+myb2+mymu).M()'
dm_a = '(m_bb-m_sv)/m_sv'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b2ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb2)'
dR_b2mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb2)'
dR_bb = 'ROOT::Math::VectorUtil::DeltaR(myb1,myb2)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
dR_b2tt = 'ROOT::Math::VectorUtil::DeltaR(myb2,mytt)'
dR_bbele = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,myele)'
dR_bbmu = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,mymu)'
dR_aa = 'ROOT::Math::VectorUtil::DeltaR(mytt,myb1+myb2)'

mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
mT_b2 = 'sqrt(pow(myb2.Pt()+mymet.Pt(),2)-pow(myb2.Px()+mymet.Px(),2)-pow(myb2.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(len(files18)):
    files18[i] = files18[i].Define('myele', myele)\
    .Define('mymu', mymu)\
    .Define('mymet', mymet)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('myb2', myb2)\
    .Define('m_b1tt', m_b1tt)\
    .Define('m_b2tt', m_b2tt)\
    .Define('m_bbtt', m_bbtt)\
    .Define('m_bb', m_bb)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('m_b2ele', m_b2ele)\
    .Define('m_b2mu', m_b2mu)\
    .Define('m_bbele', m_bbele)\
    .Define('m_bbmu', m_bbmu)\
    .Define('dm_a', dm_a)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b2ele', dR_b2ele)\
    .Define('dR_b2mu', dR_b2mu)\
    .Define('dR_bb', dR_bb)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('dR_b2tt', dR_b2tt)\
    .Define('dR_aa', dR_aa)\
    .Define('dR_bbele', dR_bbele)\
    .Define('dR_bbmu', dR_bbmu)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('mT_b2', mT_b2)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['pt_1', 'pt_2', 'm_b1tt', 'm_b2tt', 'm_bbtt', 'm_bb', 'm_b1ele', 'm_b1mu', 'm_b2ele', 'm_b2mu', 'dm_a',\
                'm_bbele', 'm_bbmu', 'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b2ele', 'dR_b2mu', 'dR_bb', 'dR_b1tt', 'dR_b2tt', 'dR_aa',\
                'dR_bbele','dR_bbmu', 'mT_ele', 'mT_mu', 'mT_b1', 'mT_b2', 'Dzeta',\
                'bpt_deepflavour_1', 'bpt_deepflavour_2']

## Muon energy scale uncertainties

### Muon ES values for both MC and embedded

In [ ]:
#All years up
muonES = ['1.004','1.009','1.027']
m_sv_shifted = 'm_sv_muonESUp'

In [ ]:
#All years down
muonES = ['0.996','0.991','0.973']
m_sv_shifted = 'm_sv_muonESDown'

### Muon ES values for data_obs (no shifting)

In [ ]:
#All years down
muonES = ['1','1','1']
m_sv_shifted = 'm_sv'

### Define variables with shifted muon ES for 1 bjet

In [ ]:
myele = 'ROOT::Math::PtEtaPhiMVector(pt_1,eta_1,phi_1,m_1)'
myrawmu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
mymu = 'if(gen_match_2==2 or gen_match_2==4){\
          if(fabs(eta_2)<1.2){return muonES0*myrawmu;}\
          else if(fabs(eta_2)<2.1){return muonES1*myrawmu;}\
          else{return muonES2*myrawmu;}\
        }\
        else{return myrawmu;}'
myrawmet = 'ROOT::Math::PtEtaPhiMVector(met,0,metphi,0)'
mymet = 'myrawmet + myrawmu - mymu'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mymu+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv_shifted)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_1,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
m_btt = '(mytt+myb1).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
tt_pt = 'mytt.Pt()'
tt_eta = 'mytt.Eta()'

mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(len(files18)):
    files18[i] = files18[i].Define('myele', myele)\
    .Define('myrawmu', myrawmu)\
    .Define('muonES0', muonES[0])\
    .Define('muonES1', muonES[1])\
    .Define('muonES2', muonES[2])\
    .Define('mymu', mymu)\
    .Define('mupt', 'mymu.Pt()')\
    .Define('myrawmet', myrawmet)\
    .Define('mymet', mymet)\
    .Define('m_sv_shifted', m_sv_shifted)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('m_btt', m_btt)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('tt_pt', tt_pt)\
    .Define('tt_eta', tt_eta)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['pt_1', 'mupt', 'm_btt', 'm_b1ele', 'm_b1mu',\
                'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b1tt',\
                'tt_eta', 'mT_ele', 'mT_mu', 'mT_b1', 'Dzeta',\
                'bpt_deepflavour_1']

### Define variables with shifted muon ES for 2 bjets

In [ ]:
myele = 'ROOT::Math::PtEtaPhiMVector(pt_1,eta_1,phi_1,m_1)'
myrawmu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
mymu = 'if(gen_match_2==2 or gen_match_2==4){\
          if(fabs(eta_2)<1.2){return muonES0*myrawmu;}\
          else if(fabs(eta_2)<2.1){return muonES1*myrawmu;}\
          else{return muonES2*myrawmu;}\
        }\
        else{return myrawmu;}'
myrawmet = 'ROOT::Math::PtEtaPhiMVector(met,0,metphi,0)'
mymet = 'myrawmet + myrawmu - mymu'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mymu+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv_shifted)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_1,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
myb2 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_2,beta_deepflavour_2,bphi_deepflavour_2,bm_deepflavour_2)'
m_b1tt = '(mytt+myb1).M()'
m_b2tt = '(mytt+myb2).M()'
m_bbtt = '(mytt+myb1+myb2).M()'
m_bb = '(myb1+myb2).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
m_b2ele = '(myele+myb2).M()'
m_b2mu = '(mymu+myb2).M()'
m_bbele = '(myb1+myb2+myele).M()'
m_bbmu = '(myb1+myb2+mymu).M()'
dm_a = '(m_bb-m_sv_shifted)/m_sv_shifted'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b2ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb2)'
dR_b2mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb2)'
dR_bb = 'ROOT::Math::VectorUtil::DeltaR(myb1,myb2)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
dR_b2tt = 'ROOT::Math::VectorUtil::DeltaR(myb2,mytt)'
dR_bbele = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,myele)'
dR_bbmu = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,mymu)'
dR_aa = 'ROOT::Math::VectorUtil::DeltaR(mytt,myb1+myb2)'

mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
mT_b2 = 'sqrt(pow(myb2.Pt()+mymet.Pt(),2)-pow(myb2.Px()+mymet.Px(),2)-pow(myb2.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(len(files18)):
    files18[i] = files18[i].Define('myele', myele)\
    Define('myrawmu', myrawmu)\
    .Define('muonES0', muonES[0])\
    .Define('muonES1', muonES[1])\
    .Define('muonES2', muonES[2])\
    .Define('mymu', mymu)\
    .Define('mupt', 'mymu.Pt()')\
    .Define('myrawmet', myrawmet)\
    .Define('mymet', mymet)\
    .Define('m_sv_shifted', m_sv_shifted)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('myb2', myb2)\
    .Define('m_b1tt', m_b1tt)\
    .Define('m_b2tt', m_b2tt)\
    .Define('m_bbtt', m_bbtt)\
    .Define('m_bb', m_bb)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('m_b2ele', m_b2ele)\
    .Define('m_b2mu', m_b2mu)\
    .Define('m_bbele', m_bbele)\
    .Define('m_bbmu', m_bbmu)\
    .Define('dm_a', dm_a)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b2ele', dR_b2ele)\
    .Define('dR_b2mu', dR_b2mu)\
    .Define('dR_bb', dR_bb)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('dR_b2tt', dR_b2tt)\
    .Define('dR_aa', dR_aa)\
    .Define('dR_bbele', dR_bbele)\
    .Define('dR_bbmu', dR_bbmu)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('mT_b2', mT_b2)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['pt_1', 'mupt', 'm_b1tt', 'm_b2tt', 'm_bbtt', 'm_bb', 'm_b1ele', 'm_b1mu', 'm_b2ele', 'm_b2mu', 'dm_a',\
                'm_bbele', 'm_bbmu', 'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b2ele', 'dR_b2mu', 'dR_bb', 'dR_b1tt', 'dR_b2tt', 'dR_aa',\
                'dR_bbele','dR_bbmu', 'mT_ele', 'mT_mu', 'mT_b1', 'mT_b2', 'Dzeta',\
                'bpt_deepflavour_1', 'bpt_deepflavour_2']

## Electron energy scale uncertainties

### Electron ES values for embedded (not for MC)

In [ ]:
#All years up
eleES = ['1.005','1.0125']
m_sv_shifted = 'm_sv_ESCALEUP'

In [ ]:
#All years down
eleES = ['0.995','0.9875']
m_sv_shifted = 'm_sv_ESCALEDOWN'

### Electron ES values for MC (not for embedded)

In [ ]:
#All years up
pt_1_shifted = ['pt_1_ScaleUp']
m_sv_shifted = 'm_sv_ESCALEUP'

In [ ]:
#All years down
pt_1_shifted = ['pt_1_ScaleDown']
m_sv_shifted = 'm_sv_ESCALEDOWN'

### Electron ES values for data_obs (no shifting)

In [ ]:
#All years down
eleES = ['1','1']
m_sv_shifted = 'm_sv'

### Define variables with shifted electron ES for 1 bjet (embedded and data)

In [ ]:
myrawele = 'ROOT::Math::PtEtaPhiMVector(pt_1,eta_1,phi_1,m_1)'
myele = 'if(fabs(eta_1)<1.479){return eleES0*myrawele;}\
         else{return eleES1*myrawele;}'
mymu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
myrawmet = 'ROOT::Math::PtEtaPhiMVector(met,0,metphi,0)'
mymet = 'myrawmet + myrawele - myele'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mymu+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv_shifted)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_1,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
m_btt = '(mytt+myb1).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
tt_pt = 'mytt.Pt()'
tt_eta = 'mytt.Eta()'

mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(len(files18)):
    files18[i] = files18[i].Define('myrawele', myrawele)\
    .Define('eleES0', eleES[0])\
    .Define('eleES1', eleES[1])\
    .Define('myele', myele)\
    .Define('elept', 'myele.Pt()')\
    .Define('mymu', mymu)\
    .Define('myrawmet', myrawmet)\
    .Define('mymet', mymet)\
    .Define('m_sv_shifted', m_sv_shifted)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('m_btt', m_btt)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('tt_pt', tt_pt)\
    .Define('tt_eta', tt_eta)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['elept', 'pt_2', 'm_btt', 'm_b1ele', 'm_b1mu',\
                'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b1tt',\
                'tt_eta', 'mT_ele', 'mT_mu', 'mT_b1', 'Dzeta',\
                'bpt_deepflavour_1']

### Define variables with shifted electron ES for 2 bjets (embedded and data)

In [ ]:
myrawele = 'ROOT::Math::PtEtaPhiMVector(pt_1,eta_1,phi_1,m_1)'
myele = 'if(fabs(eta_1)<1.479){return eleES0*myrawele;}\
         else{return eleES1*myrawele;}'
mymu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
myrawmet = 'ROOT::Math::PtEtaPhiMVector(met,0,metphi,0)'
mymet = 'myrawmet + myrawele - myele'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mymu+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv_shifted)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_1,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
myb2 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_2,beta_deepflavour_2,bphi_deepflavour_2,bm_deepflavour_2)'
m_b1tt = '(mytt+myb1).M()'
m_b2tt = '(mytt+myb2).M()'
m_bbtt = '(mytt+myb1+myb2).M()'
m_bb = '(myb1+myb2).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
m_b2ele = '(myele+myb2).M()'
m_b2mu = '(mymu+myb2).M()'
m_bbele = '(myb1+myb2+myele).M()'
m_bbmu = '(myb1+myb2+mymu).M()'
dm_a = '(m_bb-m_sv_shifted)/m_sv_shifted'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b2ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb2)'
dR_b2mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb2)'
dR_bb = 'ROOT::Math::VectorUtil::DeltaR(myb1,myb2)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
dR_b2tt = 'ROOT::Math::VectorUtil::DeltaR(myb2,mytt)'
dR_bbele = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,myele)'
dR_bbmu = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,mymu)'
dR_aa = 'ROOT::Math::VectorUtil::DeltaR(mytt,myb1+myb2)'

mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
mT_b2 = 'sqrt(pow(myb2.Pt()+mymet.Pt(),2)-pow(myb2.Px()+mymet.Px(),2)-pow(myb2.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(len(files18)):
    files18[i] = files18[i].Define('myrawele', myrawele)\
    .Define('eleES0', eleES[0])\
    .Define('eleES1', eleES[1])\
    .Define('myele', myele)\
    .Define('elept', 'myele.Pt()')\
    .Define('mymu', mymu)\
    .Define('myrawmet', myrawmet)\
    .Define('mymet', mymet)\
    .Define('m_sv_shifted', m_sv_shifted)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('myb2', myb2)\
    .Define('m_b1tt', m_b1tt)\
    .Define('m_b2tt', m_b2tt)\
    .Define('m_bbtt', m_bbtt)\
    .Define('m_bb', m_bb)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('m_b2ele', m_b2ele)\
    .Define('m_b2mu', m_b2mu)\
    .Define('m_bbele', m_bbele)\
    .Define('m_bbmu', m_bbmu)\
    .Define('dm_a', dm_a)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b2ele', dR_b2ele)\
    .Define('dR_b2mu', dR_b2mu)\
    .Define('dR_bb', dR_bb)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('dR_b2tt', dR_b2tt)\
    .Define('dR_aa', dR_aa)\
    .Define('dR_bbele', dR_bbele)\
    .Define('dR_bbmu', dR_bbmu)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('mT_b2', mT_b2)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['elept', 'pt_2', 'm_b1tt', 'm_b2tt', 'm_bbtt', 'm_bb', 'm_b1ele', 'm_b1mu', 'm_b2ele', 'm_b2mu', 'dm_a',\
                'm_bbele', 'm_bbmu', 'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b2ele', 'dR_b2mu', 'dR_bb', 'dR_b1tt', 'dR_b2tt', 'dR_aa',\
                'dR_bbele','dR_bbmu', 'mT_ele', 'mT_mu', 'mT_b1', 'mT_b2', 'Dzeta',\
                'bpt_deepflavour_1', 'bpt_deepflavour_2']

### Define variables with shifted electron ES for 1 bjet (MC)

In [ ]:
myele = 'ROOT::Math::PtEtaPhiMVector(pt_1_shifted,eta_1,phi_1,m_1)'
mymu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
myrawmet = 'ROOT::Math::PtEtaPhiMVector(met,0,metphi,0)'
mymet = 'myrawmet + myrawele - myele'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mymu+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv_shifted)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_1,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
m_btt = '(mytt+myb1).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
tt_pt = 'mytt.Pt()'
tt_eta = 'mytt.Eta()'

mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(len(files18)):
    files18[i] = files18[i].Define('pt_1_shifted', pt_1_shifted)\
    .Define('myele', myele)\
    .Define('elept', 'myele.Pt()')\
    .Define('mymu', mymu)\
    .Define('myrawmet', myrawmet)\
    .Define('mymet', mymet)\
    .Define('m_sv_shifted', m_sv_shifted)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('m_btt', m_btt)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('tt_pt', tt_pt)\
    .Define('tt_eta', tt_eta)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['elept', 'pt_2', 'm_btt', 'm_b1ele', 'm_b1mu',\
                'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b1tt',\
                'tt_eta', 'mT_ele', 'mT_mu', 'mT_b1', 'Dzeta',\
                'bpt_deepflavour_1']

### Define variables with shifted electron ES for 2 bjets (MC)

In [ ]:
myele = 'ROOT::Math::PtEtaPhiMVector(pt_1_shifted,eta_1,phi_1,m_1)'
mymu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
myrawmet = 'ROOT::Math::PtEtaPhiMVector(met,0,metphi,0)'
mymet = 'myrawmet + myrawele - myele'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mymu+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv_shifted)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_1,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
myb2 = 'ROOT::Math::PtEtaPhiMVector(bpt_deepflavour_2,beta_deepflavour_2,bphi_deepflavour_2,bm_deepflavour_2)'
m_b1tt = '(mytt+myb1).M()'
m_b2tt = '(mytt+myb2).M()'
m_bbtt = '(mytt+myb1+myb2).M()'
m_bb = '(myb1+myb2).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
m_b2ele = '(myele+myb2).M()'
m_b2mu = '(mymu+myb2).M()'
m_bbele = '(myb1+myb2+myele).M()'
m_bbmu = '(myb1+myb2+mymu).M()'
dm_a = '(m_bb-m_sv_shifted)/m_sv_shifted'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b2ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb2)'
dR_b2mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb2)'
dR_bb = 'ROOT::Math::VectorUtil::DeltaR(myb1,myb2)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
dR_b2tt = 'ROOT::Math::VectorUtil::DeltaR(myb2,mytt)'
dR_bbele = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,myele)'
dR_bbmu = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,mymu)'
dR_aa = 'ROOT::Math::VectorUtil::DeltaR(mytt,myb1+myb2)'

mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
mT_b2 = 'sqrt(pow(myb2.Pt()+mymet.Pt(),2)-pow(myb2.Px()+mymet.Px(),2)-pow(myb2.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(len(files18)):
    files18[i] = files18[i].Define('pt_1_shifted', pt_1_shifted)\
    .Define('myele', myele)\
    .Define('elept', 'myele.Pt()')\
    .Define('mymu', mymu)\
    .Define('myrawmet', myrawmet)\
    .Define('mymet', mymet)\
    .Define('m_sv_shifted', m_sv_shifted)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('myb2', myb2)\
    .Define('m_b1tt', m_b1tt)\
    .Define('m_b2tt', m_b2tt)\
    .Define('m_bbtt', m_bbtt)\
    .Define('m_bb', m_bb)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('m_b2ele', m_b2ele)\
    .Define('m_b2mu', m_b2mu)\
    .Define('m_bbele', m_bbele)\
    .Define('m_bbmu', m_bbmu)\
    .Define('dm_a', dm_a)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b2ele', dR_b2ele)\
    .Define('dR_b2mu', dR_b2mu)\
    .Define('dR_bb', dR_bb)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('dR_b2tt', dR_b2tt)\
    .Define('dR_aa', dR_aa)\
    .Define('dR_bbele', dR_bbele)\
    .Define('dR_bbmu', dR_bbmu)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('mT_b2', mT_b2)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['elept', 'pt_2', 'm_b1tt', 'm_b2tt', 'm_bbtt', 'm_bb', 'm_b1ele', 'm_b1mu', 'm_b2ele', 'm_b2mu', 'dm_a',\
                'm_bbele', 'm_bbmu', 'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b2ele', 'dR_b2mu', 'dR_bb', 'dR_b1tt', 'dR_b2tt', 'dR_aa',\
                'dR_bbele','dR_bbmu', 'mT_ele', 'mT_mu', 'mT_b1', 'mT_b2', 'Dzeta',\
                'bpt_deepflavour_1', 'bpt_deepflavour_2']

## Jet/JER/recoil/UES uncertainties

### Variables with uncertainties already shifted in ntuples

In [ ]:
#Up
m_sv_shifted = ['m_sv_JetAbsoluteUp','m_sv_JetAbsoluteyearUp','m_sv_JetBBEC1Up','m_sv_JetBBEC1yearUp','m_sv_JetEC2Up','m_sv_JetEC2yearUp','m_sv_JetFlavorQCDUp','m_sv_JetHFUp','m_sv_JetHFyearUp','m_sv_JetRelativeBalUp','m_sv_JetRelativeSampleUp','m_sv_JERUp','m_sv_ResponseUp','m_sv_ResolutionUp','m_sv_UESUp']
met_shifted = ['met_JetAbsoluteUp','met_JetAbsoluteyearUp','met_JetBBEC1Up','met_JetBBEC1yearUp','met_JetEC2Up','met_JetEC2yearUp','met_JetFlavorQCDUp','met_JetHFUp','met_JetHFyearUp','met_JetRelativeBalUp','met_JetRelativeSampleUp','met_JERUp','met_responseUp','met_resolutionUp','met_UESUp']
metphi_shifted = ['metphi_JetAbsoluteUp','metphi_JetAbsoluteyearUp','metphi_JetBBEC1Up','metphi_JetBBEC1yearUp','metphi_JetEC2Up','metphi_JetEC2yearUp','metphi_JetFlavorQCDUp','metphi_JetHFUp','metphi_JetHFyearUp','metphi_JetRelativeBalUp','metphi_JetRelativeSampleUp','metphi_JERUp','metphi_responseUp','metphi_resolutionUp','metphi_UESUp']
bpt_1_shifted = ['bpt_deepflavour_JetAbsoluteUp_1','bpt_deepflavour_JetAbsoluteyearUp_1','bpt_deepflavour_JetBBEC1Up_1','bpt_deepflavour_JetBBEC1yearUp_1','bpt_deepflavour_JetEC2Up_1','bpt_deepflavour_JetEC2yearUp_1','bpt_deepflavour_JetFlavorQCDUp_1','bpt_deepflavour_JetHFUp_1','bpt_deepflavour_JetHFyearUp_1','bpt_deepflavour_JetRelativeBalUp_1','bpt_deepflavour_JetRelativeSampleUp_1','bpt_deepflavour_JERUp_1','bpt_deepflavour_1','bpt_deepflavour_1','bpt_deepflavour_1']
bpt_2_shifted = ['bpt_deepflavour_JetAbsoluteUp_2','bpt_deepflavour_JetAbsoluteyearUp_2','bpt_deepflavour_JetBBEC1Up_2','bpt_deepflavour_JetBBEC1yearUp_2','bpt_deepflavour_JetEC2Up_2','bpt_deepflavour_JetEC2yearUp_2','bpt_deepflavour_JetFlavorQCDUp_2','bpt_deepflavour_JetHFUp_2','bpt_deepflavour_JetHFyearUp_2','bpt_deepflavour_JetRelativeBalUp_2','bpt_deepflavour_JetRelativeSampleUp_2','bpt_deepflavour_JERUp_2','bpt_deepflavour_2','bpt_deepflavour_2','bpt_deepflavour_2']

n = 5 #(0-14)
m_sv_shifted = m_sv_shifted[n]
met_shifted = met_shifted[n]
metphi_shifted = metphi_shifted[n]
bpt_1_shifted = bpt_1_shifted[n]
bpt_2_shifted = bpt_2_shifted[n]
print(m_sv_shifted)
print(met_shifted)
print(metphi_shifted)
print(bpt_1_shifted)
print(bpt_2_shifted)

In [ ]:
#Down
m_sv_shifted = ['m_sv_JetAbsoluteDown','m_sv_JetAbsoluteyearDown','m_sv_JetBBEC1Down','m_sv_JetBBEC1yearDown','m_sv_JetEC2Down','m_sv_JetEC2yearDown','m_sv_JetFlavorQCDDown','m_sv_JetHFDown','m_sv_JetHFyearDown','m_sv_JetRelativeBalDown','m_sv_JetRelativeSampleDown','m_sv_JERDown','m_sv_ResponseDown','m_sv_ResolutionDown','m_sv_UESDown']
met_shifted = ['met_JetAbsoluteDown','met_JetAbsoluteyearDown','met_JetBBEC1Down','met_JetBBEC1yearDown','met_JetEC2Down','met_JetEC2yearDown','met_JetFlavorQCDDown','met_JetHFDown','met_JetHFyearDown','met_JetRelativeBalDown','met_JetRelativeSampleDown','met_JERDown','met_responseDown','met_resolutionDown','met_UESDown']
metphi_shifted = ['metphi_JetAbsoluteDown','metphi_JetAbsoluteyearDown','metphi_JetBBEC1Down','metphi_JetBBEC1yearDown','metphi_JetEC2Down','metphi_JetEC2yearDown','metphi_JetFlavorQCDDown','metphi_JetHFDown','metphi_JetHFyearDown','metphi_JetRelativeBalDown','metphi_JetRelativeSampleDown','metphi_JERDown','metphi_responseDown','metphi_resolutionDown','metphi_UESDown']
bpt_1_shifted = ['bpt_deepflavour_JetAbsoluteDown_1','bpt_deepflavour_JetAbsoluteyearDown_1','bpt_deepflavour_JetBBEC1Down_1','bpt_deepflavour_JetBBEC1yearDown_1','bpt_deepflavour_JetEC2Down_1','bpt_deepflavour_JetEC2yearDown_1','bpt_deepflavour_JetFlavorQCDDown_1','bpt_deepflavour_JetHFDown_1','bpt_deepflavour_JetHFyearDown_1','bpt_deepflavour_JetRelativeBalDown_1','bpt_deepflavour_JetRelativeSampleDown_1','bpt_deepflavour_JERDown_1','bpt_deepflavour_1','bpt_deepflavour_1','bpt_deepflavour_1']
bpt_2_shifted = ['bpt_deepflavour_JetAbsoluteDown_2','bpt_deepflavour_JetAbsoluteyearDown_2','bpt_deepflavour_JetBBEC1Down_2','bpt_deepflavour_JetBBEC1yearDown_2','bpt_deepflavour_JetEC2Down_2','bpt_deepflavour_JetEC2yearDown_2','bpt_deepflavour_JetFlavorQCDDown_2','bpt_deepflavour_JetHFDown_2','bpt_deepflavour_JetHFyearDown_2','bpt_deepflavour_JetRelativeBalDown_2','bpt_deepflavour_JetRelativeSampleDown_2','bpt_deepflavour_JERDown_2','bpt_deepflavour_2','bpt_deepflavour_2','bpt_deepflavour_2']

n = 5 #(0-14)
m_sv_shifted = m_sv_shifted[n]
met_shifted = met_shifted[n]
metphi_shifted = metphi_shifted[n]
bpt_1_shifted = bpt_1_shifted[n]
bpt_2_shifted = bpt_2_shifted[n]
print(m_sv_shifted)
print(met_shifted)
print(metphi_shifted)
print(bpt_1_shifted)
print(bpt_2_shifted)

### Define variables with jet uncertainties for 1 bjet

In [ ]:
myele = 'ROOT::Math::PtEtaPhiMVector(pt_1,eta_1,phi_1,m_1)'
mymu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
mymet = 'ROOT::Math::PtEtaPhiMVector(met_shifted,0,metphi_shifted,0)'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mym+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv_shifted)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_1_shifted,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
m_btt = '(mytt+myb1).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
tt_pt = 'mytt.Pt()'
tt_eta = 'mytt.Eta()'

mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(2, 3):
    allfiles18[i] = allfiles18[i].Define('myele', myele)\
    .Define('mymu', mymu)\
    .Define('m_sv_shifted', m_sv_shifted)\
    .Define('met_shifted', met_shifted)\
    .Define('metphi_shifted', metphi_shifted)\
    .Define('bpt_1_shifted', bpt_1_shifted)\
    .Define('mymet', mymet)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('m_btt', m_btt)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('tt_pt', tt_pt)\
    .Define('tt_eta', tt_eta)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['pt_1', 'pt_2', 'm_btt', 'm_b1ele', 'm_b1mu',\
                'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b1tt',\
                'tt_eta', 'mT_ele', 'mT_mu', 'mT_b1', 'Dzeta',\
                'bpt_1_shifted']

### Define variables with jet uncertainties for 2 bjets

In [ ]:
myele = 'ROOT::Math::PtEtaPhiMVector(pt_1,eta_1,phi_1,m_1)'
mymu = 'ROOT::Math::PtEtaPhiMVector(pt_2,eta_2,phi_2,m_2)'
mymet = 'ROOT::Math::PtEtaPhiMVector(met_shifted,0,metphi_shifted,0)'
mytt = 'ROOT::Math::PtEtaPhiMVector((myele+mymu+mymet).Pt(),(myele+mymu+mymet).Eta(),(myele+mymu+mymet).Phi(),m_sv_shifted)'
myb1 = 'ROOT::Math::PtEtaPhiMVector(bpt_1_shifted,beta_deepflavour_1,bphi_deepflavour_1,bm_deepflavour_1)'
myb2 = 'ROOT::Math::PtEtaPhiMVector(bpt_2_shifted,beta_deepflavour_2,bphi_deepflavour_2,bm_deepflavour_2)'
m_b1tt = '(mytt+myb1).M()'
m_b2tt = '(mytt+myb2).M()'
m_bbtt = '(mytt+myb1+myb2).M()'
m_bb = '(myb1+myb2).M()'
m_b1ele = '(myele+myb1).M()'
m_b1mu = '(mymu+myb1).M()'
m_b2ele = '(myele+myb2).M()'
m_b2mu = '(mymu+myb2).M()'
m_bbele = '(myb1+myb2+myele).M()'
m_bbmu = '(myb1+myb2+mymu).M()'
dm_a = '(m_bb-m_sv_shifted)/m_sv_shifted'
dR_tt = 'ROOT::Math::VectorUtil::DeltaR(myele,mymu)'
dR_b1ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb1)'
dR_b1mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb1)'
dR_b2ele = 'ROOT::Math::VectorUtil::DeltaR(myele,myb2)'
dR_b2mu = 'ROOT::Math::VectorUtil::DeltaR(mymu,myb2)'
dR_bb = 'ROOT::Math::VectorUtil::DeltaR(myb1,myb2)'
dR_b1tt = 'ROOT::Math::VectorUtil::DeltaR(myb1,mytt)'
dR_b2tt = 'ROOT::Math::VectorUtil::DeltaR(myb2,mytt)'
dR_bbele = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,myele)'
dR_bbmu = 'ROOT::Math::VectorUtil::DeltaR(myb1+myb2,mymu)'
dR_aa = 'ROOT::Math::VectorUtil::DeltaR(mytt,myb1+myb2)'

# define transverse masses mT and D_zeta
mT_ele = 'sqrt(pow(myele.Pt()+mymet.Pt(),2)-pow(myele.Px()+mymet.Px(),2)-pow(myele.Py()+mymet.Py(),2))'
mT_mu = 'sqrt(pow(mymu.Pt()+mymet.Pt(),2)-pow(mymu.Px()+mymet.Px(),2)-pow(mymu.Py()+mymet.Py(),2))'
mT_b1 = 'sqrt(pow(myb1.Pt()+mymet.Pt(),2)-pow(myb1.Px()+mymet.Px(),2)-pow(myb1.Py()+mymet.Py(),2))'
mT_b2 = 'sqrt(pow(myb2.Pt()+mymet.Pt(),2)-pow(myb2.Px()+mymet.Px(),2)-pow(myb2.Py()+mymet.Py(),2))'
norm_zeta = 'sqrt(pow(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt(),2)+pow(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt(),2))'
x_zeta = '(myele.Px()/myele.Pt()+mymu.Px()/mymu.Pt())/norm_zeta'
y_zeta = '(myele.Py()/myele.Pt()+mymu.Py()/mymu.Pt())/norm_zeta'
p_zeta_mis = 'mymet.Px()*x_zeta+mymet.Py()*y_zeta'
pzeta_vis = '(myele.Px()+mymu.Px())*x_zeta+(myele.Py()+mymu.Py())*y_zeta'
Dzeta = 'p_zeta_mis-0.85*pzeta_vis'

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
for i in range(2, 3):
    allfiles18[i] = allfiles18[i].Define('myele', myele)\
    .Define('mymu', mymu)\
    .Define('m_sv_shifted', m_sv_shifted)\
    .Define('met_shifted', met_shifted)\
    .Define('metphi_shifted', metphi_shifted)\
    .Define('bpt_1_shifted', bpt_1_shifted)\
    .Define('bpt_2_shifted', bpt_2_shifted)\
    .Define('mymet', mymet)\
    .Define('mytt', mytt)\
    .Define('myb1', myb1)\
    .Define('myb2', myb2)\
    .Define('m_b1tt', m_b1tt)\
    .Define('m_b2tt', m_b2tt)\
    .Define('m_bbtt', m_bbtt)\
    .Define('m_bb', m_bb)\
    .Define('m_b1ele', m_b1ele)\
    .Define('m_b1mu', m_b1mu)\
    .Define('m_b2ele', m_b2ele)\
    .Define('m_b2mu', m_b2mu)\
    .Define('m_bbele', m_bbele)\
    .Define('m_bbmu', m_bbmu)\
    .Define('dm_a', dm_a)\
    .Define('dR_tt', dR_tt)\
    .Define('dR_b1ele', dR_b1ele)\
    .Define('dR_b1mu', dR_b1mu)\
    .Define('dR_b2ele', dR_b2ele)\
    .Define('dR_b2mu', dR_b2mu)\
    .Define('dR_bb', dR_bb)\
    .Define('dR_b1tt', dR_b1tt)\
    .Define('dR_b2tt', dR_b2tt)\
    .Define('dR_aa', dR_aa)\
    .Define('dR_bbele', dR_bbele)\
    .Define('dR_bbmu', dR_bbmu)\
    .Define('mT_ele', mT_ele)\
    .Define('mT_mu', mT_mu)\
    .Define('mT_b1', mT_b1)\
    .Define('mT_b2', mT_b2)\
    .Define('norm_zeta', norm_zeta)\
    .Define('x_zeta', x_zeta)\
    .Define('y_zeta', y_zeta)\
    .Define('p_zeta_mis', p_zeta_mis)\
    .Define('pzeta_vis', pzeta_vis)\
    .Define('Dzeta', Dzeta)

feature_list = ['pt_1', 'pt_2', 'm_b1tt', 'm_b2tt', 'm_bbtt', 'm_bb', 'm_b1ele', 'm_b1mu', 'm_b2ele', 'm_b2mu', 'dm_a',\
                'm_bbele', 'm_bbmu', 'dR_tt', 'dR_b1ele', 'dR_b1mu', 'dR_b2ele', 'dR_b2mu', 'dR_bb', 'dR_b1tt', 'dR_b2tt', 'dR_aa',\
                'dR_bbele','dR_bbmu', 'mT_ele', 'mT_mu', 'mT_b1', 'mT_b2', 'Dzeta',\
                'bpt_1_shifted', 'bpt_2_shifted']

## Re-compute with shifted variables and save outputs

In [ ]:
#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
#######change 1b/2b
for i in range(2,3):
    allfiles18[i] = pd.DataFrame(allfiles18[i].AsNumpy(feature_list))
    allfiles18[i] = allfiles18[i].values
    allfiles18[i] = savedscaler_mt1b.transform(allfiles18[i])
    
print('done pre-processing...')

#######data=range(0,1); embedded=range(1,2); MC=range(2,len(allfiles18))
#######change 18/17/16
#######change 1b/2b
#######change tree names
for i in range(2,3):
    y_pred = savedmodel_mt1b.predict(allfiles18[i])
    y_pred = np.array(y_pred, dtype = [('NN1b', np.float32)])
    array2root(y_pred, filename = outpaths18[i], treename = 'emu_tree_NN', mode = 'update')
    
print('done writing outputs!')